In [1]:
using Laplacians
using ProfileView

In [2]:
n = 1000;
a = wtedChimera(n, 1);

In [3]:
include("../../../src/johnlind.jl")
@time reff = johnlind(a, 0.5);

  5.021142 seconds (3.75 M allocations: 888.846 MB, 2.76% gc time)


In [4]:
n = 20000;
a = wtedChimera(n, 1);

In [6]:
@time reff = johnlind(a, 5.0);

  1.374442 seconds (1.61 M allocations: 747.173 MB, 14.90% gc time)


In [18]:
# Profile.clear()
# @profile reff = johnlind(a, 0.5);
# ProfileView.view()

In [7]:
levScore = copy(a);
for i in 1:length(a.nzval)
    levScore.nzval[i] = a.nzval[i] * reff.nzval[i]
end
sum(levScore.nzval) / 2

20019.831590528414

In [11]:
include("../../../src/johnlind.jl")
reffMin = copy(a);
reffMin.nzval = ones(length(a.nzval)) * 2 * n
reffMax = copy(a);
reffMax.nzval = zeros(length(a.nzval))

@time for test in 1:10
    reff = johnlind(a, 1.0)
    for i in 1:length(reff.nzval)
        reffMin.nzval[i] = min(reffMin.nzval[i], reff.nzval[i])
        reffMax.nzval[i] = max(reffMax.nzval[i], reff.nzval[i])
    end
end

ratioMax = 0
for i in 1:length(reff.nzval)
    ratioMax = max(ratioMax, reffMax.nzval[i] / reffMin.nzval[i])
end
println(ratioMax)

221.352078 seconds (28.49 M allocations: 124.216 GB, 7.67% gc time)
4.375484221715097


In [10]:
inv = pinv(full(lap(a)));
totLevScore = 0
pos = 1
for i in 1:length(a.nzval)
    while a.colptr[pos + 1] <= i
        pos = pos + 1
    end

    p = a.rowval[i]
    q = pos
    w = a.nzval[i]
    
    if p > q
        edge = zeros(a.n)
        edge[p] = 1
        edge[q] = -1
        totLevScore += w * edge' * inv * edge
    end
end
totLevScore

1-element Array{Float64,1}:
 99.0